In [1]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON

list_of_prefix="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
"""

class Triple:
    def __init__(self, sub, pre, obj):
        self.sub = sub
        self.pre = pre
        self.obj = obj

    def get_subject(self):

        return self.sub

    def get_predicate(self):
        return self.pre

    def get_object(self):
        return self.obj


class Approx:
    def __init__(self, init_query,cost):
        self.query=SPARQL(init_query)
        self.cost=cost
        self.triples_to_query=[]
        self.original_query_t=[[], [], []]       
        self.approx_queries=dict()
        self.final_list = []
        self.xxx  = []
        self.get_triples()

        


    def get_triples(self):
        for s in self.query.abbr_triple_list:
            self.triples_to_query.append(
                Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
            self.original_query_t[0].append(
                str(self.triples_to_query[-1].get_subject()))
            self.original_query_t[1].append(
                str(self.triples_to_query[-1].get_predicate()))
            self.original_query_t[2].append(
                str(self.triples_to_query[-1].get_object()))
        self.approximate()

    def approximate(self):
        for c in range(self.cost+1):
            value=[]
            if c == 0:
                value.append(self.triples_to_query)
                # build_query(value)
                self.execute_query(value, c)
            else:
                pre_approx=self.approx_queries[str(c-1)]
                # self.xxx.append(pre_approx)
                for triple in pre_approx:
                    new_approx=self.calculate(triple, c) #each term in the list is set to calculate
                    self.execute_query(new_approx, c)
                    # build_query(new_approx)
                    value += new_approx
            self.approx_queries[str(c)] = value
    
        

    def calculate(self, triples, cost):
        values=[]

        def left_insertion( index):
            list_of_approx=triples[:]
            list_of_approx.insert(index, (Triple(list_of_approx[index].get_subject(), f"?p{cost}", f"?o{cost}")))
            list_of_approx[index + 1]=Triple(f"?o{cost}", list_of_approx[index+1].get_predicate(), list_of_approx[index+1].get_object())
            return list_of_approx

        def right_insertion(index):
            list_of_approx=triples[:]
            list_of_approx.insert(index+1, (Triple(f"?o{cost}", f"?p{cost}", list_of_approx[index].get_object())))
            list_of_approx[index]=Triple(list_of_approx[index].get_subject(), list_of_approx[index].get_predicate(), f"?o{cost}")
            return list_of_approx

        def sub(index):
            list_of_approx=triples[:]
            list_of_approx[index]=(Triple(triples[index].get_subject(), f"?p{cost}", triples[index].get_object()))
            return list_of_approx
    


        for triple in triples:
            
            index=triples.index(triple)
            if (triple.get_subject() in self.original_query_t[0]):
                values.append(left_insertion(index))
            if (triple.get_predicate() in self.original_query_t[1]):
                values.append(sub(index))
            if (triple.get_object() in self.original_query_t[2]):
                values.append(right_insertion(index))
        return values

    

    def execute_query(self, queries, cost):

        def wr_my_list(l1list,cost):  
                file_path = 'test4.txt'
                with open(file_path, 'a') as file:
                    for l1 in zip(*l1list):
                        for l2 in l1:
                            if l2 not in self.final_list:
                                self.final_list.append(l2)
                                file.write(str(l2) + " " + str(cost) + "\n" )

        def build_query(queries):
                sparql_query=f"{self.query.intent} distinct{self.query.former.strip(self.query.intent)} where {{\n"
                for triple in queries:
                    sparql_query += f"{triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
                sparql_query += "}\n LIMIT 25"
                return (list_of_prefix + sparql_query)

        for query in queries:
            query_to_execute = build_query(query)
            sparql=SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
            sparql.setQuery(query_to_execute)
            sparql.setReturnFormat(JSON)
            results=sparql.query().convert()

            index=results['head']['vars']

            ind_reslut=[]
            result_list=[]


            for y in index:
                for x1 in results['results']['bindings']:
                    ind_reslut.append(x1[y]['value'])
                result_list.append(ind_reslut)
                ind_reslut=[]
                
            wr_my_list(result_list, cost)




# pass the final list as a argument, first part empty then every other cycle it will have the previous shit, we can also do that with the dic

In [2]:
triple = Triple("<http://dbpedia.org/resource/Albert_Einstein>", "<http://dbpedia.org/ontology/birthPlace>", "?place")
query = f"""
SELECT ?place
WHERE {{
   dbr:Albert_Einstein dbo:birthPlace ?place .
    
}}
"""
ll= []
x =  Approx(query,1)
for y in x.final_list:
    # if(y not in ll):
    #     ll.append(y)
    # print(type(y))
    print(f"{y} {x.final_list.index(y)}")  
# x.approx_queries["1"][0]

# if x:
#     for i in x.final_list:
#         print(i.get_subject(), i.get_predicate(), i.get_object())
#     # print( x.final_list[0].get_subject(), x.final_list[0].get_predicate(), x.final_list[0].get_object())
# else:
#     print("nope")
# x.xxx.get_subject()

http://dbpedia.org/resource/German_Empire 0
http://dbpedia.org/resource/Ulm 1
http://dbpedia.org/resource/Kingdom_of_Württemberg 2
http://dbpedia.org/resource/Austria-Hungary 3
http://dbpedia.org/resource/Munich 4
http://dbpedia.org/resource/Bad_Buchau 5
http://dbpedia.org/resource/Vojvodina 6
http://dbpedia.org/resource/Zurich 7
http://dbpedia.org/resource/Novi_Sad 8
http://dbpedia.org/resource/Württemberg 9
http://dbpedia.org/resource/Cannstatt 10
http://dbpedia.org/resource/Switzerland 11
http://dbpedia.org/resource/Canton_of_Zurich 12
http://dbpedia.org/resource/Maschwanden 13
http://dbpedia.org/resource/Weimar_Germany 14
http://dbpedia.org/resource/Bern 15
http://dbpedia.org/resource/German_Confederation 16
http://dbpedia.org/resource/Saxe-Weimar-Eisenach 17
http://dbpedia.org/resource/Magdala,_Germany 18
http://dbpedia.org/resource/Austro-Hungarian_Empire 19
http://dbpedia.org/resource/Titel 20
http://dbpedia.org/resource/Brooklyn 21
http://www.w3.org/2002/07/owl#Thing 22
http://

In [1]:
class relax:
    def __init__(self, init_query, cost):
        self.query = SPARQL(init_query)
        self.cost = cost
        self.triples_to_query = []
        self.original_query_t = [[], [], []]
        self.approx_queries = dict()
        self.final_list = []
        self.xxx = []
        self.get_triples()

    def get_triples(self):
        for s in self.query.abbr_triple_list:
            self.triples_to_query.append(
                Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
            self.original_query_t[0].append(
                str(self.triples_to_query[-1].get_subject()))
            self.original_query_t[1].append(
                str(self.triples_to_query[-1].get_predicate()))
            self.original_query_t[2].append(
                str(self.triples_to_query[-1].get_object()))
        self.relax()

    def relax(self):
        for c in range(self.cost+1):
            value = []
            if c == 0:
                value.append(self.triples_to_query)
                # build_query(value)
                self.execute_query(value, c)
            else:
                pre_approx = self.approx_queries[str(c-1)]
                # self.xxx.append(pre_approx)
                for triple in pre_approx:
                    new_approx = self.calculate(triple,)

                    self.execute_query(new_approx, c)
                    # build_query(new_approx)
                    value += new_approx
            self.approx_queries[str(c)] = value

    def calculate(self, triples):
        values = []
        for triple in triples:
            if (str(triple.get_predicate()) == "rdfs:$subPropertyOf" or str(triple.get_predicate()) == "<http://www.w3.org/2000/01/rdf-schema#subPropertyOf>"):
                if self.r1(triple):
                    triple_copy = triples[:]
                    triple_copy[triples.index(triple)] = self.r1(triple)
                    values.append(triple_copy)
            if self.r2(triple):
                triple_copy = triples[:]
                triple_copy[triples.index(triple)] = self.r2(triple)
                values.append(triple_copy)
            if ((str(triple.get_predicate()) == "rdfs:subClassOf" or str(triple.get_predicate()) == "<http://www.w3.org/2000/01/rdf-schema#subClassOf>")):
                if self.r3(triple):      
                    triple_copy = triples[:]
                    triple_copy[triples.index(triple)] = self.r3(triple)
                    values.append(triple_copy)
            if ((str(triple.get_predicate()) == "rdf:type" or str(triple.get_predicate()) == " <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>")):
                if self.r4(triple):
                    triple_copy = triples[:]
                    triple_copy[triples.index(triple)] = self.r4(triple)
                    values.append(triple_copy)
            if self.r5(triple):
                triple_copy = triples[:]
                triple_copy[triples.index(triple)] = self.r5(triple)
                values.append(triple_copy)
            if self.r6(triple):
                triple_copy = triples[:]
                triple_copy[triples.index(triple)] = self.r6(triple)
                values.append(triple_copy)
        return values


    def r1(self, triple):
        query = f"""
            SELECT ?a WHERE {{
                ?a rdfs:subPropertyOf {triple.get_subject()}.
            }}
        """
        a = self.endpoint(list_of_prefix + query)
        if a:
            return (Triple(f"<{a[0]}>", triple.get_predicate(), triple.get_object()))

    def r2(self, triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                {triple.get_predicate()} rdfs:subPropertyOf ?b.
            }}
        """
        b = self.endpoint(list_of_prefix + query)
        if b:
            return (Triple(triple.get_subject(), f"<{b[0]}>", triple.get_object()))

    def r3(self, triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                ?a rdfs:subClassOf {triple.get_subject()}.
            }}
        """
        a = self.endpoint(list_of_prefix + query)
        if a:
            return (Triple(f"<{a[0]}>", triple.get_predicate(), triple.get_object()))

    def r4(self, triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                {triple.get_object()} rdfs:subClassOf ?b.
            }}
        """
        b = self.endpoint(list_of_prefix + query)
        if b:
            return (Triple(triple.get_subject(), triple.get_predicate(), f"<{b[0]}>"))

    def r5(self, triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?c WHERE {{
                {triple.get_predicate()} rdfs:domain ?c.
            }}
        """
        c = self.endpoint(list_of_prefix + query)
        if c:
            return (Triple(triple.get_subject(), "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>", f"<{c[0]}>"))

    def r6(self, triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?d WHERE {{
                {triple.get_predicate()} rdfs:range ?d.
            }}
        """
        d = self.endpoint(list_of_prefix + query)
        if d:
            return (Triple(triple.get_object(), "<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>", f"<{d[0]}>"))



    def endpoint(self, query):
        sparql = SPARQLWrapper(
            "https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        index = results['head']['vars']

        ind_reslut = []
        result_list = []

        for y in index:
            for x1 in results['results']['bindings']:
                ind_reslut.append(x1[y]['value'])
            result_list += (ind_reslut)
            ind_reslut = []
        return (result_list)

    def execute_query(self, queries: list, cost):

        def wr_my_list(l1list, cost):
            file_path = 'test2.txt'
            with open(file_path, 'a') as file:
                for l1 in zip(*l1list):
                    for l2 in l1:
                        self.final_list.append(l2)
                        file.write(str(l2) + " " + str(cost) + "\n")

        def build_query(queries):
            sparql_query = f"{self.query.intent} distinct{self.query.former.strip(self.query.intent)} where {{\n"
            for triple in queries:
                sparql_query += f"{triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
            sparql_query += "}\n LIMIT 25"
            return (list_of_prefix + sparql_query)

        print()
        print(cost)
        print()
        for query in queries:
            query_to_execute = build_query(query)
            print(query_to_execute)
            # sparql=SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
            # sparql.setQuery(query_to_execute)
            # sparql.setReturnFormat(JSON)
            # results=sparql.query().convert()

            # index=results['head']['vars']

            # ind_reslut=[]
            # result_list=[]

            # for y in index:
            #     for x1 in results['results']['bindings']:
            #         ind_reslut.append(x1[y]['value'])
            #     result_list.append(ind_reslut)
            #     ind_reslut=[]

            # wr_my_list(result_list, cost)
